# Analysis

In [41]:
import os

# 1. Select potential folder
#potential_folder = "2025_Sharifi"
potential_folder = "2022_Mahata"
#potential_folder = "Zhou04_eam_alloy"
#potential_folder = "Cu1_eam_fs"

# 2. Pick simulation parameters: composition, temperature and shear strain rate
x = 1.00  # Cu fraction
T = 1800  # Temperature (K)
xyrate_1_per_s = 1e9  # Shear rate (1/s)
xyrate_str = f"{xyrate_1_per_s:.0e}".replace("+0", "").replace("+", "") # For path

# Directory and file path formatting
path = os.path.join(
    "/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux",
    potential_folder, f"x_{x:.2f}/{T}K/{xyrate_str}/"
)
os.makedirs(path, exist_ok=True)

## Plot the average velocity profile

In [42]:
from ovito.io import import_file
dump_file = os.path.join(path, "dump_nemd_analyze")
pipeline = import_file(dump_file)
data = pipeline.compute(0)  # 0 = first frame

# Get the y box length
length_y = data.cell.matrix[1, 1]
print("Y box length:", length_y)

Y box length: 53.68288698020543


In [43]:
import pandas as pd
import numpy as np
from dfttk.plotly_format import plot_format

# Read the velocity profile file
vx_file = os.path.join(path, "vx_profile_run_avg.txt")

# Using pandas - clean and simple
df = pd.read_csv(vx_file, skiprows=4, sep='\s+', names=['Chunk', 'Coord1', 'Ncount', 'vx'])

# Remove boundary chunks where Ncount is 0
df_filtered = df[df['Ncount'] > 0].copy()

# Access the data for plotting
coord1 = df_filtered['Coord1'].values * length_y
vx = df_filtered['vx'].values

# Fit a linear model: coord1 = slope * vx + intercept
coeffs = np.polyfit(vx, coord1, 1)  # coeffs[0]: slope, coeffs[1]: intercept
slope, intercept = coeffs[0], coeffs[1]

# Generate points for the fit line
fit_x = np.linspace(np.min(vx), np.max(vx), 100)
fit_y = slope * fit_x + intercept

# Calculate R-squared
fit_y_data = slope * vx + intercept
ss_res = np.sum((coord1 - fit_y_data) ** 2)  # Sum of squares of residuals
ss_tot = np.sum((coord1 - np.mean(coord1)) ** 2)  # Total sum of squares
r_squared = 1 - (ss_res / ss_tot)

# Plot coord1 vs. vx using plotly
import plotly.graph_objects as go
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(
    x=vx, 
    y=coord1, 
    mode='markers',
    name='Data',
    marker=dict(size=8)
))

# Add linear fit line
fig.add_trace(go.Scatter(
    x=fit_x,
    y=fit_y,
    mode='lines',
    name=f'Linear Fit (R² = {r_squared:.4f})',
    line=dict(dash='dash', color='red', width=2)
))

plot_format(fig, xtitle='v<sub>x</sub> (Å·ps⁻¹)', ytitle='Y (Å)')

# Add annotation with fit equation and R²
fig.add_annotation(
    x=0.05, y=0.95,
    xref="paper", yref="paper",
    text=f"y = {slope:.4f}x + {intercept:.4f}<br>R² = {r_squared:.4f}",
    showarrow=False,
    font=dict(size=12),
    bgcolor="white",
    bordercolor="black",
    borderwidth=1
)

fig.show()

# Print the results
print(f"Linear fit equation: y = {slope:.4f}x + {intercept:.4f}")
print(f"R-squared: {r_squared:.4f}")

Linear fit equation: y = 1160.9951x + 26.9511
R-squared: 0.9761


## Calculate Viscosity

In [44]:
# Read the -pxy running average file
neg_pxy_file = os.path.join(path, "pxy_run_avg.txt")

# Using pandas - clean and simple
df = pd.read_csv(neg_pxy_file, skiprows=2, sep='\s+', names=['Timestep', 'Neg_pxy'])

# Take the final neg_pxy value
pxy_run_avg = df['Neg_pxy'].values[-1]

# Calculate viscosity 
vx_run_avg = 1/slope
viscosity = (pxy_run_avg/vx_run_avg)*0.0001 # Convert bar.ps to mPa.s 
print(f"Viscosity: {viscosity:.2f} mPa.s")

Viscosity: 2.92 mPa.s
